# Load and Encode

Here we download congressional voting records (and some legislator metadata) and encode it for use with machine learning algorithms.

Some command line work is needed before processing with Python. 

Congressional voting records are publically available online and can be scraped with 
[this project](https://github.com/unitedstates/congress/wiki/votes). 
I download the repo and setup an environment with Anaconda to use it; note that Python2 is needed to use the repo but the rest of this and subsequent notebooks are in Python3.
 
```shell
git clone https://github.com/unitedstates/congress.git
cd congress
conda create --name congress python=2.7
source activate congress
pip install -r requirements.txt
```

Then download data for some sessions. A list of congress and session numbers are available [here](https://www.senate.gov/reference/Sessions/sessionDates.htm)

```shell
./run votes --congress=115 --session=2017
./run votes --congress=114 --session=2016
./run votes --congress=114 --session=2015
./run votes --congress=113 --session=2014
./run votes --congress=113 --session=2013
./run votes --congress=112 --session=2012
./run votes --congress=112 --session=2011
./run votes --congress=111 --session=2010
./run votes --congress=111 --session=2009
```

Example terminal output:

```
Going to fetch 910 votes from congress #112 session 2012
[h327-112.2012] Skipping: vote was vacated
Errors for 0.
Skipped 1.
Saved data for 909.
```

We'll also download metadata for legislators and store it in a dataframe. The metadata is sourced from 
[this repo](https://github.com/unitedstates/congress-legislators) and can be downloaded with:

```shell
cd ..
wget -P data/meta https://raw.githubusercontent.com/unitedstates/congress-legislators/master/legislators-historical.yaml
wget -P data/meta https://raw.githubusercontent.com/unitedstates/congress-legislators/master/legislators-current.yaml
```

In [74]:
from datetime import date
import numpy as np
import pandas as pd
import json
import yaml
import glob
import os

## Voting Records

Each bill voted on is in the form a JSON file (there is also an XML file which we will not be using). The function below takes one of these files as input and outputs a 
one-row dataframe with columns corresponding to legislators that voted and elements filled with their raw votes like 'Yea' or 'Nay'.

In [8]:
def mk_row(file):
    '''
    Takes the path of a json file containing voting data for a single proposal.
    
    Returns a one-row df with columns as voter IDs and elements filled with their vote.
    The index is set as the unique proposal id e.g. 'h112-112.2011'. 
    
    json structure is like
    {
        metadata1: foo
        metadata2: bar
        votes: {
            Nay: [{} {}]
            Aye: [{} {}]
        }
    }
    '''
    
    ## Open and extract json
    with open(file) as f:
        data = json.load(f)
    
    ## Grab vote results
    ids = []
    casts = []
    votes = data['votes']
    for key in votes.keys():
        results = votes[key]
        idsk = [voter['id'] for voter in results if voter != 'VP']
        castsk = [key for i in range(len(idsk))]
        ids.extend(idsk)
        casts.extend(castsk)

    ## Use the voteid as a unique index
    voteid = data['vote_id']
    
    return pd.DataFrame(data = [casts], columns = ids, index = [voteid])

Here's a test case. The row index `s8-112.2011` refers to the 8th bill in the Senate of Congress 112 in the session year 2011. The Congress and session numbers are important as they will be used later to filter the dataframe down to specific sessions.

Note that the column names are not the legislators' actual names but rather some kind of ID code. The metadata processed below contains the information to map ID codes to names as well as other information like party affiliation.

In [2]:
test_file = 'congress/data/112/votes/2011/s8/data.json'
df = mk_row(test_file)
df.head()

,S213,S277,S253,S332,S331,S172,S051,S166,S057,S131,...,S284,S314,S303,S351,S325,S326,S299,S315,S318,S247
s8-112.2011,Nay,Nay,Nay,Nay,Nay,Nay,Nay,Nay,Nay,Nay,...,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea


Now, apply the function to all the downloaded files.

In [11]:
## Load all voting data
files = glob.glob('./congress/data/**/votes/**/*.json', recursive=True)
df_list = [mk_row(file) for file in files]
df = pd.concat(df_list, axis=0)

## Save
df.to_csv('data/merged_raw.tsv', sep = '\t')

## Preview
print('Found voting data for {} proposals'.format(len(files)))
print('Final merged shape: {}'.format(df.shape))
df.head()

Found voting data for 9781 proposals
Final merged shape: (9781, 923)


,A000014,A000022,A000055,A000210,A000358,A000361,A000362,A000363,A000364,A000365,...,W000822,Y000031,Y000033,Y000062,Y000063,Y000064,Y000065,Y000066,Z000017,Z000018
h593-111.2009,Yea,Yea,Nay,Yea,Nay,Nay,Nay,Nay,Nay,Nay,...,NaN,Nay,Nay,Yea,NaN,NaN,NaN,NaN,NaN,NaN
h1-111.2009,Present,Present,Present,Present,Present,Present,Present,Present,Present,Present,...,NaN,Present,Present,Present,NaN,NaN,NaN,NaN,NaN,NaN
h10-111.2009,Present,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea,...,NaN,Yea,Yea,Yea,NaN,NaN,NaN,NaN,NaN,NaN
h100-111.2009,Aye,Aye,No,Aye,No,No,Aye,Aye,Aye,No,...,NaN,No,No,Aye,NaN,NaN,NaN,NaN,NaN,NaN
h101-111.2009,No,No,Aye,No,Aye,Aye,Aye,Aye,No,Aye,...,NaN,Aye,Aye,No,NaN,NaN,NaN,NaN,NaN,NaN


This above operation takes awhile; use this cell to avoid redoing the whole operation.

In [46]:
df = pd.read_table('data/merged_raw.tsv', index_col=0, dtype='str')
df.head()

,A000014,A000022,A000055,A000210,A000358,A000361,A000362,A000363,A000364,A000365,...,W000822,Y000031,Y000033,Y000062,Y000063,Y000064,Y000065,Y000066,Z000017,Z000018
h593-111.2009,Yea,Yea,Nay,Yea,Nay,Nay,Nay,Nay,Nay,Nay,...,NaN,Nay,Nay,Yea,NaN,NaN,NaN,NaN,NaN,NaN
h1-111.2009,Present,Present,Present,Present,Present,Present,Present,Present,Present,Present,...,NaN,Present,Present,Present,NaN,NaN,NaN,NaN,NaN,NaN
h10-111.2009,Present,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea,Yea,...,NaN,Yea,Yea,Yea,NaN,NaN,NaN,NaN,NaN,NaN
h100-111.2009,Aye,Aye,No,Aye,No,No,Aye,Aye,Aye,No,...,NaN,No,No,Aye,NaN,NaN,NaN,NaN,NaN,NaN
h101-111.2009,No,No,Aye,No,Aye,Aye,Aye,Aye,No,Aye,...,NaN,Aye,Aye,No,NaN,NaN,NaN,NaN,NaN,NaN


Some of the votes are not simple Yea/Nay votes. Some are names when Congress votes on issues like electing the House Speaker. For simplicity, in the next cells, the raw data is filtered down to only the Yea/Nay/Aye/No/Not Voting records. This excludes an insignificant number of bills (about 30 across the 9 years examined).

In [48]:
## Find all votes
votes_unique = set(pd.unique(df.values.ravel()))
print('Found the following votes: {}\n'.format(votes_unique))

## Map invalid votes to NaN
valid_votes = set(['Aye', 'Yea', 'Nay', 'No', 'Not Voting', np.nan])
invalid_votes = votes_unique - valid_votes
null_map = dict(zip(invalid_votes, [np.nan for i in invalid_votes]))
df.replace(null_map, inplace = True)

## Filter out rows which are now all NaN
print('Shape before filtering: {}'.format(df.shape))
df = df.loc[~((df.isna()) | (df == 'Not Voting')).all(axis = 1)]
print('Shape after filtering: {}'.format(df.shape))

Found the following votes: {nan, 'David Walker', 'Gohmert', 'Pelosi', 'Nay', 'Gowdy', 'Cardoza', 'Aye', 'Lewis (GA)', 'Labrador', 'Yoho', 'Hon. Rand Paul', 'Dingell', 'Present', 'Cantor', 'Colin Powell', 'Hoyer', 'DeFazio', 'Not Voting', 'Guilty', 'Boehner', 'Yea', 'Lewis', 'Duncan (SC)', 'McCarthy', 'Not Guilty', 'No', 'Amash', 'Allen West', 'Kaptur', 'Webster (FL)', 'Ryan (WI)', 'Costa', 'Hon. Jeff Sessions', 'Cooper', 'Ryan (OH)', 'Shuler', 'Jordan'}

Shape before filtering: (9781, 923)
Shape after filtering: (9751, 923)


With the filtered data, we now map all the Aye's, Yea's, Nay's, No's, and Not Voting records to simpler 'Yes'/'No'/'Neutral' codes. Everything else, such as if a legislator does not appear in the voting records for a bill, is left as `np.NaN`

In [49]:
## Encoding scheme
yes_code = 'Yes'
no_code = 'No'
neutral_code = 'Neutral'
vote_map = {
    'Aye': yes_code,
    'Yea': yes_code,
    'Nay': no_code,
    'No': no_code,
    'Not Voting': neutral_code,
}

## Update
df.replace(vote_map, inplace = True)
df.head()

,A000014,A000022,A000055,A000210,A000358,A000361,A000362,A000363,A000364,A000365,...,W000822,Y000031,Y000033,Y000062,Y000063,Y000064,Y000065,Y000066,Z000017,Z000018
h593-111.2009,Yes,Yes,No,Yes,No,No,No,No,No,No,...,NaN,No,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN
h10-111.2009,NaN,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,NaN,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
h100-111.2009,Yes,Yes,No,Yes,No,No,Yes,Yes,Yes,No,...,NaN,No,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN
h101-111.2009,No,No,Yes,No,Yes,Yes,Yes,Yes,No,Yes,...,NaN,Yes,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN
h102-111.2009,Yes,Yes,Yes,Yes,Neutral,Yes,Yes,Yes,Yes,Yes,...,NaN,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN


Now we filter the frame to different congress/session subsets, one-hot encode, and save for later processing.

In [50]:
## Make list of (congress, session) tuples
pairs = [
    (115, 2017),
    (114, 2016),
    (114, 2015),
    (113, 2014),
    (113, 2013),
    (112, 2012),
    (112, 2011),
    (111, 2010),
    (111, 2009)
]

for congress, session in pairs:
    print(congress, session)
    
    ## Filter
    df_subset = df.filter(like='{}.{}'.format(congress, session), axis=0)
    df_subset = df_subset.dropna(axis = 1, how = 'all')
    print(df_subset.shape)
    
    ## Encode
    df_onehot = pd.get_dummies(df_subset.transpose(), dummy_na = True)
    print(df_onehot.shape)
    print()
    
    df_onehot.to_csv('data/onehot/congress{}_session{}.tsv'.format(congress, session), sep = '\t')
    
    
df_onehot.head()

115 2017
(1007, 542)
(542, 3887)

114 2016
(783, 538)
(538, 3057)

114 2015
(1039, 537)
(537, 4006)

113 2014
(928, 539)
(539, 3577)

113 2013
(930, 542)
(542, 3593)

112 2012
(908, 541)
(541, 3593)

112 2011
(1179, 540)
(540, 4582)

111 2010
(955, 551)
(551, 3552)

111 2009
(1383, 550)
(550, 5245)



,h593-111.2009_Neutral,h593-111.2009_No,h593-111.2009_Yes,h593-111.2009_nan,h10-111.2009_Neutral,h10-111.2009_No,h10-111.2009_Yes,h10-111.2009_nan,h100-111.2009_Neutral,h100-111.2009_No,...,s75-111.2009_Yes,s75-111.2009_nan,s76-111.2009_Neutral,s76-111.2009_No,s76-111.2009_Yes,s76-111.2009_nan,s77-111.2009_Neutral,s77-111.2009_No,s77-111.2009_Yes,s77-111.2009_nan
A000014,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
A000022,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
A000055,0,1,0,0,0,0,1,0,0,1,...,0,1,0,0,0,1,0,0,0,1
A000210,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
A000358,0,1,0,0,0,0,1,0,0,1,...,0,1,0,0,0,1,0,0,0,1


## Metadata

The metadata has useful information such as mapping from legislator ID codes to their full names. We read all metadata (for current and historical legislators) into a single frame.

In [75]:
with open('data/meta/legislators-historical.yaml', 'r', encoding='utf_8') as f:
    dfh = pd.io.json.json_normalize(yaml.load(f))
    print('Finished historical.')
    
with open('deta/meta/legislators-current.yaml', 'r', encoding='utf_8') as f:
    dfc = pd.io.json.json_normalize(yaml.load(f))
    print('Finished current.\n')


namesc = set(dfc.columns)
namesh = set(dfh.columns)
print('In current but not historical: {}'.format(namesc-namesh))
print('In historical but not current: {}\n'.format(namesh-namesc))

## Merge them, keeping all data.
dfmeta = dfc.append(dfh)
dfmeta.set_index('id.bioguide', drop = False, inplace = True)
dfmeta.index.names = ['ID']
print('Shape of historical|current|merged: {}|{}|{}'.format(dfc.shape, dfh.shape, dfmeta.shape))
print('Columns in merged metadata frame: {}'.format(dfmeta.columns))

print(dfmeta.shape)
dfmeta.head()

Finished historical.
Finished current.

In current but not historical: set()
In historical but not current: {'id.house_history_alternate', 'id.bioguide_previous'}

Shape of historical|current|merged: (536, 28)|(11870, 30)|(12406, 30)
Columns in merged metadata frame: Index(['bio.birthday', 'bio.gender', 'bio.religion', 'family',
       'id.ballotpedia', 'id.bioguide', 'id.bioguide_previous', 'id.cspan',
       'id.fec', 'id.google_entity_id', 'id.govtrack', 'id.house_history',
       'id.house_history_alternate', 'id.icpsr', 'id.lis', 'id.maplight',
       'id.opensecrets', 'id.thomas', 'id.votesmart', 'id.wikidata',
       'id.wikipedia', 'leadership_roles', 'name.first', 'name.last',
       'name.middle', 'name.nickname', 'name.official_full', 'name.suffix',
       'other_names', 'terms'],
      dtype='object')
(12406, 30)


,bio.birthday,bio.gender,bio.religion,family,id.ballotpedia,id.bioguide,id.bioguide_previous,id.cspan,id.fec,id.google_entity_id,...,id.wikipedia,leadership_roles,name.first,name.last,name.middle,name.nickname,name.official_full,name.suffix,other_names,terms
ID,,,,,,,,,,,,,,,,,,,,,
B000944,1952-11-09,M,Lutheran,NaN,Sherrod Brown,B000944,NaN,5051.0,"[H2OH13033, S6OH00163]",kg:/m/034s80,...,Sherrod Brown,NaN,Sherrod,Brown,NaN,NaN,Sherrod Brown,NaN,NaN,"[{'type': 'rep', 'start': '1993-01-05', 'end':..."
C000127,1958-10-13,F,Roman Catholic,NaN,Maria Cantwell,C000127,NaN,26137.0,"[S8WA00194, H2WA01054]",kg:/m/01x68t,...,Maria Cantwell,NaN,Maria,Cantwell,NaN,NaN,Maria Cantwell,NaN,NaN,"[{'type': 'rep', 'start': '1993-01-05', 'end':..."
C000141,1943-10-05,M,Jewish,NaN,Ben Cardin,C000141,NaN,4004.0,"[H6MD03177, S6MD03177]",kg:/m/025k3k,...,Ben Cardin,NaN,Benjamin,Cardin,L.,NaN,Benjamin L. Cardin,NaN,NaN,"[{'type': 'rep', 'start': '1987-01-06', 'end':..."
C000174,1947-01-23,M,Presbyterian,NaN,Tom Carper,C000174,NaN,663.0,[S8DE00079],kg:/m/01xw7t,...,Tom Carper,NaN,Thomas,Carper,Richard,NaN,Thomas R. Carper,NaN,NaN,"[{'type': 'rep', 'start': '1983-01-03', 'end':..."
C001070,1960-04-13,M,NaN,NaN,"Bob Casey, Jr.",C001070,NaN,47036.0,[S6PA00217],kg:/m/047ymw,...,Bob Casey Jr.,NaN,Robert,Casey,P.,Bob,"Robert P. Casey, Jr.",Jr.,NaN,"[{'type': 'sen', 'start': '2007-01-04', 'end':..."


Now add on columns for party, state, and chamber affiliation which are currently embedded in the `terms` column. Each entry in `terms` is a list of dictionaries. Each dictionary corresponds to a range of dates and has info for the congressman during those dates such as party affiliation.

The function below loops through the list of dictionaries and searches for one corresponding to a given session year (technically, it searches for the political affiliation on July 4th of the session year which is an arbitrary choice). If it exists, the query will be taken from that year. Otherwise, the most common party/state affiliations will be used instead (in case of a tie, one is picked randomly).

In [76]:
def get_terms_query(terms, session, query):
    '''
    Return the info queried of US Congressmen for a given congressional session. 
    If the congressman was not in office during the session, return his/her 
    most common affiliation.
    
    terms: List of dictionaries representing a congressman with at least 'start', 'end', and 'party' values.
    session: The session year (not necessary coresponding to calendar year) to search for (int or str)
    query: The key to search for like 'state' or 'party'.

    Returns a string like 'Democrat' or 'AZ'
    '''
    query = str(query)
    queries = []
    session_date = date(int(session), 7, 4) # July 4th, [session year]
    
    ## Check if congressmen was in office during session
    for term in terms:
        if query not in term.keys(): continue
        queries.append(term[query])
        term_start = date(*[int(i) for i in term['start'].split('-')])
        term_end = date(*[int(i) for i in term['end'].split('-')])
        if term_start < session_date < term_end:
            return term[query]
        
    ## Otherwise, return most common affiliation.
    if len(queries) == 0: return 'Unknown'
    return(max(queries, key = queries.count))

Although it's unlikely that someone switches parties from year-to-year, it has happened. So, rather than having one column each for party and state, we'll make multiple for each year. That is, there will be `party2011`, `party2012`, etc. columns.

In [77]:
for _, session in pairs:
    session = str(session)
    dfmeta['party'+session] = [get_terms_query(terms, session, 'party') for terms in dfmeta['terms']]
    dfmeta['state'+session] = [get_terms_query(terms, session, 'state') for terms in dfmeta['terms']]
    dfmeta['chamber'+session] = [get_terms_query(terms, session, 'type') for terms in dfmeta['terms']]
print(dfmeta.shape)
dfmeta.head()

(12406, 57)


,bio.birthday,bio.gender,bio.religion,family,id.ballotpedia,id.bioguide,id.bioguide_previous,id.cspan,id.fec,id.google_entity_id,...,chamber2012,party2011,state2011,chamber2011,party2010,state2010,chamber2010,party2009,state2009,chamber2009
ID,,,,,,,,,,,,,,,,,,,,,
B000944,1952-11-09,M,Lutheran,NaN,Sherrod Brown,B000944,NaN,5051.0,"[H2OH13033, S6OH00163]",kg:/m/034s80,...,sen,Democrat,OH,sen,Democrat,OH,sen,Democrat,OH,sen
C000127,1958-10-13,F,Roman Catholic,NaN,Maria Cantwell,C000127,NaN,26137.0,"[S8WA00194, H2WA01054]",kg:/m/01x68t,...,sen,Democrat,WA,sen,Democrat,WA,sen,Democrat,WA,sen
C000141,1943-10-05,M,Jewish,NaN,Ben Cardin,C000141,NaN,4004.0,"[H6MD03177, S6MD03177]",kg:/m/025k3k,...,sen,Democrat,MD,sen,Democrat,MD,sen,Democrat,MD,sen
C000174,1947-01-23,M,Presbyterian,NaN,Tom Carper,C000174,NaN,663.0,[S8DE00079],kg:/m/01xw7t,...,sen,Democrat,DE,sen,Democrat,DE,sen,Democrat,DE,sen
C001070,1960-04-13,M,NaN,NaN,"Bob Casey, Jr.",C001070,NaN,47036.0,[S6PA00217],kg:/m/047ymw,...,sen,Democrat,PA,sen,Democrat,PA,sen,Democrat,PA,sen


Finally, some of the legislators are curiously missing the `name.official_full` column which we will definitely be needing. Below, the missing values are filled using values from other columns.

In [78]:
dfmeta.loc[dfmeta['name.official_full'].isna(), 'name.official_full'] = dfmeta['name.first'] + ' ' + dfmeta['name.last']
dfmeta.to_csv('data/meta/meta_info.tsv', sep = '\t')